In [1]:
# time series analysis

In [2]:
# 위키피디아 조회수 예측 예제
# 1. 위키피디아 페이지의 일일 조회수 데이터를 내려받습니다.
# 2. 데이터를 보면 이상치가 있습니다. 상위 5% 백분위보다 높은 조회수는 5% 백분위 값으로 대체합니다.
# 3. 요일이 조회수에 미치는 영향이 있을 거라는 가설을 세웠습니다. 즉, 조회수가 일주일을 주기로 하는 주기성을 가진다는 가설
# 4. 길이가 일주일인 윈도우를 만들고 이 윈도우를 적용해 특징값을 추출합니다. 이 값을 이용해 다음 일주일 동안 조회수를 예측하는 회귀모델을 학습

In [1]:
import urllib, json
import pandas as pd
import numpy as np
import sklearn.linear_model, statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
START_DATE = "20131010"
END_DATE = "20161012"
WINDOW_SIZE = 7
TOPIC = "Cat"
URL_TEMPLATE = ("https://wikimedia.org/api/rest_v1"
               "/metrics/pageviews/per-article"
               "/en.wikipedia/all-access/"
               "allagents/%s/daily/%s/%s")

In [3]:
# 조회수를 불러오는 함수
def get_time_series(topic, start, end):
    url = URL_TEMPLATE % (topic, start, end)
    json_data = urllib.request.urlopen(url).read().decode('utf-8')
    data = json.loads(json_data)
    times = [rec['timestamp'] for rec in data['items']]
    values = [rec['views'] for rec in data['items']]
    times_formatted = pd.Series(times).map(
        lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:8])
    time_index = times_formatted.astype('datetime64')
    return pd.DataFrame(
        {'views': values}, index=time_index)

In [4]:
# 선형 회귀 모델을 학습하는 함수
def line_slope(ss):
    X = np.arange(len(ss)).reshape((len(ss),1))
    linear.fit(X,ss)
    return linear.coef_

In [5]:
# 선형 회귀 모델을 하나 만든다.
# 이제 모델에 다양한 데일터를 계속 적용한다.
linear = sklearn.linear_model.LinearRegression()

In [6]:
df = get_time_series(TOPIC, START_DATE, END_DATE)

HTTPError: HTTP Error 400: Bad Request

In [7]:
# 시계열 데이터 시각화
df['views'].plot()
plt.title("날짜별 조회수")
plt.show()

NameError: name 'df' is not defined

In [ ]:
# 백분위를 기준으로 이상치 제거
max_views = df['views'].quantile(0.95)
df.views[df.views > max_views] = max_views
# 7일을 주기로 데이터 분석
decomp = sm.tsa.seasonal_decompose(
    df['views'].values, freq=7)
decomp.plot()
plt.suptitle("조회수 분석 결과")
plt.show()

# 날짜별로 과거 일주일의 평균, 최댓값, 최솟값 등
# 다양한 특징을 추출, 저장
df['mean_1week'] = pd.rolling_mean(
    df['views'], WINDOW_SIZE)
df['max_1week'] = pd.rolling_max(
    df['views'], WINDOW_SIZE)
df['min_1week'] = pd.rolling_min(
    df['views'], WINDOW_SIZE)
df['slope'] = pd.rolling_apply(
    df['views'], WINDOW_SIZE, line_slope)
df['total_views_week'] = pd.rolling_sum(
    df['views'], WINDOW_SIZE)
df['day_of_week'] = df.index.astype(int) % 7
day_of_week_cols = pd.get_dummies(df['day_of_week'])
df = pd.concat([df, day_of_week_cols], axis=1)

# 예측값을 준비
df['total_views_next_week'] = \
    list(df['total_views_week'][WINDOW_SIZE:]) + \
    [np.nan for _ in range(WINDOW_SIZE)]
INDEP_VARS = ['mean_1week', 'max_1week',
              'min_1week', 'slope'] + range(6)
DEP_VAR = 'total_views_next_week'

n_records = df.dropna().shape[0]
test_data = df.dropna()[:n_records / 2]
train_data = df.dropna()[n_records / 2:]

linear.fit(
    train_data[INDEP_VARS], train_data[DEP_VAR])
test_preds_array = linear.predict(
    test_data[INDEP_VARS])
test_preds = pd.Series(
    test_preds_array, index=test_data.index)
print("예측값과 정답의 상관 계수: ", \
    test_data[DEP_VAR].corr(test_preds))